In [ ]:
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns
import shap
import tqdm
from cuml.ensemble import RandomForestClassifier
from mordred import Calculator, descriptors
from rdkit import Chem
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import average_precision_score, \
    balanced_accuracy_score, precision_recall_curve, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, learning_curve, \
    StratifiedShuffleSplit
from sklearn.pipeline import Pipeline

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
# Plot styling.
plt.style.use(['seaborn-white', 'seaborn-paper'])
plt.rc('font', family='sans-serif')
sns.set_palette('Set1')
sns.set_context('paper', font_scale=1.3)

In [ ]:
compounds = pd.read_csv('../data/compound_smiles.csv')
mols = compounds['SMILES (Canonical)'].apply(Chem.MolFromSmiles)
clss = compounds['Skin'].astype(np.int32)

In [ ]:
# Calculate features using Mordred.
mordred_calculator = Calculator(descriptors, ignore_3D=True)
# Exclude features that contain non-numeric values.
features = pd.DataFrame(mordred_calculator.pandas(mols)
                        .select_dtypes(exclude='object')
                        .astype(np.float32))

In [ ]:
class CorrelationTreshold:
    
    def __init__(self, threshold=None):
        self.threshold = threshold if threshold is not None else 1.0
    
    def fit(self, X, y=None):
        corr = np.abs(np.corrcoef(X, rowvar=False))
        self.mask = ~(np.triu(corr, k=1) > self.threshold).any(axis=1)
        return self
    
    def transform(self, X, y=None):
        return X[:, self.mask]
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X, y)
    
    def get_support(self, indices=False):
        return self.mask if not indices else np.where(self.mask)[0]

In [ ]:
n_splits = 100
test_size = 0.2

variance_threshold = 0.05
correlation_threshold = 0.95
n_trees = 1000

In [ ]:
# Repeatedly evaluate the model to estimate its performance.
accuracies_train = np.zeros(n_splits, np.float32)
accuracies_test = np.zeros(n_splits, np.float32)
average_precisions_train = np.zeros(n_splits, np.float32)
average_precisions_test = np.zeros(n_splits, np.float32)
roc_aucs_train = np.zeros(n_splits, np.float32)
roc_aucs_test = np.zeros(n_splits, np.float32)
interval = np.linspace(0, 1, 101, dtype=np.float32)
tprs_train = np.zeros((n_splits, 101), np.float32)
tprs_test = np.zeros((n_splits, 101), np.float32)
precisions_train = np.zeros((n_splits, 101), np.float32)
precisions_test = np.zeros((n_splits, 101), np.float32)
best_max_depth = np.zeros(n_splits, np.uint8)
for i, (train_index, test_index) in tqdm.tqdm(
        enumerate(
            StratifiedShuffleSplit(
                n_splits, test_size=test_size, random_state=42).split(
                features.values, clss)),
        desc='Cross-validation', total=n_splits):
    features_train = features.values[train_index]
    features_test = features.values[test_index]
    clss_train, clss_test = clss[train_index], clss[test_index]
    
    grid_search = GridSearchCV(
        Pipeline([
            ('variance_threshold', VarianceThreshold(variance_threshold)),
            ('correlation_threshold', CorrelationTreshold(
                correlation_threshold)),
            ('classify', RandomForestClassifier(
                n_estimators=n_trees, random_state=42))]),
        param_grid={'classify__max_depth': np.arange(2, 8, 1)},
        cv=StratifiedShuffleSplit(n_splits, test_size=test_size,
                                  random_state=42))
    
    grid_search.fit(features_train, clss_train)
    pred_train = grid_search.predict_proba(features_train)[:, 1]
    pred_test = grid_search.predict_proba(features_test)[:, 1]
    
    # Compute evaluation metrics on the train and test data.
    accuracies_train[i] = balanced_accuracy_score(
            clss_train, np.asarray(pred_train > 0.5, np.int))
    accuracies_test[i] = balanced_accuracy_score(
            clss_test, np.asarray(pred_test > 0.5, np.int))
    average_precisions_train[i] = average_precision_score(
        clss_train, pred_train)
    average_precisions_test[i] = average_precision_score(
        clss_test, pred_test)
    roc_aucs_train[i] = roc_auc_score(clss_train, pred_train)
    roc_aucs_test[i] = roc_auc_score(clss_test, pred_test)
    fpr_train, tpr_train, _ = roc_curve(clss_train, pred_train)
    tprs_train[i] = np.interp(interval, fpr_train, tpr_train)
    fpr_test, tpr_test, _ = roc_curve(clss_test, pred_test)
    tprs_test[i] = np.interp(interval, fpr_test, tpr_test)
    precision_train, recall_train, _ = precision_recall_curve(
        clss_train, pred_train)
    precisions_train[i] = np.interp(
        interval, recall_train[::-1], precision_train[::-1])
    precision_test, recall_test, _ = precision_recall_curve(
        clss_test, pred_test)
    precisions_test[i] = np.interp(
        interval, recall_test[::-1], precision_test[::-1])
    
    # Save optimal hyperparameter(s).
    best_max_depth[i] = (grid_search.best_estimator_
                         .named_steps['classify'].max_depth)

In [ ]:
with open('eval.pickle', 'wb') as pf:
    pickle.dump([accuracies_train, accuracies_test, average_precisions_train,
                 average_precisions_test, roc_aucs_train, roc_aucs_test,
                 interval, tprs_train, tprs_test, precisions_train,
                 precisions_test, best_max_depth], pf)

In [ ]:
with open('eval.pickle', 'rb') as pf:
    accuracies_train, accuracies_test, average_precisions_train, \
        average_precisions_test, roc_aucs_train, roc_aucs_test, \
        interval, tprs_train, tprs_test, precisions_train, precisions_test, \
        best_max_depth = pickle.load(pf)

In [ ]:
stats = {'accuracy_train': np.mean(accuracies_train),
         'accuracy_std_train': np.std(accuracies_train),
         'average_precision_train': np.mean(average_precisions_train),
         'average_precision_std_train': np.std(average_precisions_train),
         'roc_auc_train': np.mean(roc_aucs_train),
         'roc_auc_std_train': np.std(roc_aucs_train),
         'tpr_mean_train': np.mean(tprs_train, axis=0),
         'tpr_std_train': np.std(tprs_train, axis=0),
         'precision_mean_train': np.mean(precisions_train, axis=0),
         'precision_std_train': np.std(precisions_train, axis=0),
        
         'accuracy_test': np.mean(accuracies_test),
         'accuracy_std_test': np.std(accuracies_test),
         'average_precision_test': np.mean(average_precisions_test),
         'average_precision_std_test': np.std(average_precisions_test),
         'roc_auc_test': np.mean(roc_aucs_test),
         'roc_auc_std_test': np.std(roc_aucs_test),
         'tpr_mean_test': np.mean(tprs_test, axis=0),
         'tpr_std_test': np.std(tprs_test, axis=0),
         'precision_mean_test': np.mean(precisions_test, axis=0),
         'precision_std_test': np.std(precisions_test, axis=0)}

In [ ]:
print(f'Accuracy: {stats["accuracy_test"]:.3f} ± '
      f'{stats["accuracy_std_test"]:.3f}')
print(f'AUROC: {stats["roc_auc_test"]:.3f} ± '
      f'{stats["roc_auc_std_test"]:.3f}')
print(f'Average precision: {stats["average_precision_test"]:.3f} ± '
      f'{stats["average_precision_std_test"]:.3f}')

In [ ]:
max_depth = scipy.stats.mode(best_max_depth)[0][0]
print(f'Optimal number of maximum depth (mode): {max_depth}')

In [ ]:
# Train the optimal model on the whole dataset and export.
classifier = Pipeline([
    ('variance_threshold', VarianceThreshold(variance_threshold)),
    ('correlation_threshold', CorrelationTreshold(correlation_threshold)),
    ('classify', RandomForestClassifier(
        n_estimators=n_trees, max_depth=max_depth, random_state=42))])
classifier.fit(features.values, clss)
# with open('rf.pickle', 'wb') as pf:
#     pickle.dump(classifier, pf)
with open('rf.pickle', 'rb') as pf:
    classifier = pickle.load(pf)

In [ ]:
width = 7
height = width / 1.618    # Golden ratio.
fig, axes = plt.subplots(1, 2, figsize=(width * 2, height))

ax = axes[0]
interval = np.linspace(0, 1, 101)
tpr_train = stats['tpr_mean_train']
tpr_train[0], tpr_train[-1] = 0, 1
ax.plot(interval, tpr_train,
        label=f'AUC (train) = {stats["roc_auc_train"]:.3f} '
              f'± {stats["roc_auc_std_train"]:.3f}')
ax.fill_between(interval, tpr_train - stats['tpr_std_train'],
                tpr_train + stats['tpr_std_train'], alpha=0.2)
tpr_test = stats['tpr_mean_test']
tpr_test[0], tpr_test[-1] = 0, 1
ax.plot(interval, tpr_test,
        label=f'AUC (test) = {stats["roc_auc_test"]:.3f} '
              f'± {stats["roc_auc_std_test"]:.3f}')
ax.fill_between(interval, tpr_test - stats['tpr_std_test'],
                tpr_test + stats['tpr_std_test'], alpha=0.2)
        
ax.plot([0, 1], [0, 1], c='black', ls='--')

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')

ax.legend(loc='lower right', frameon=False)
        
ax = axes[1]
precision_train = stats['precision_mean_train']
ax.plot(interval, precision_train,
        label=f'Avg precision (train) = '
              f'{stats["average_precision_train"]:.3f} ± '
              f'{stats["average_precision_std_train"]:.3f}')
ax.fill_between(interval, precision_train - stats['precision_std_train'],
                precision_train + stats['precision_std_train'], alpha=0.2)
precision_test = stats['precision_mean_test']
ax.plot(interval, precision_test,
        label=f'Avg precision (test) = '
              f'{stats["average_precision_test"]:.3f} ± '
              f'{stats["average_precision_std_test"]:.3f}')
ax.fill_between(interval, precision_test - stats['precision_std_test'],
                precision_test + stats['precision_std_test'], alpha=0.2)

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

ax.set_xlabel('Recall')
ax.set_ylabel('Precision')

ax.legend(loc='lower right', frameon=False)

sns.despine()

plt.savefig('roc.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# Feature importances.
predict_proba = lambda x: classifier.predict_proba(x)[:,1]
features_median = features.median().values.reshape((1, -1))
explainer = shap.KernelExplainer(predict_proba, features_median,
                                 link='logit')
shap_values = explainer.shap_values(features)

In [ ]:
height = 7
width = height / 1.618    # Golden ratio.

shap.summary_plot(shap_values, features, plot_type='bar',
                  plot_size=(width, height), show=False)

plt.gca().set_xlabel('Feature importance')

plt.savefig('feature_importance_global.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
height = 7
width = height / 1.618    # Golden ratio.

shap.summary_plot(shap_values, features,
                  plot_size=(width, height), show=False)

plt.gca().set_xlabel('Feature importance')

plt.savefig('feature_importance_local.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
width = 7
height = width / 1.618    # Golden ratio.
fig, ax = plt.subplots(figsize=(width, height))

shap.dependence_plot('MW', shap_values, features,
                     interaction_index=None, ax=ax)

fig.savefig('feature_importance_mw.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
train_sizes, train_scores, test_scores = learning_curve(
    classifier, features.values, clss, train_sizes=np.linspace(0.2, 1.0, 10),
    cv=StratifiedShuffleSplit(n_splits, test_size=0.2, random_state=42),
    scoring='roc_auc')

In [ ]:
width = 7
height = width / 1.618    # Golden ratio.
fig, ax = plt.subplots(figsize=(width, height))

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)
ax.plot(train_sizes, train_scores_mean, 'o-', label='Training score')
ax.fill_between(train_sizes, train_scores_mean - train_scores_std,
                train_scores_mean + train_scores_std, alpha=0.2)
ax.plot(train_sizes, test_scores_mean, 'o-', label='Test score')
ax.fill_between(train_sizes, test_scores_mean - test_scores_std,
                test_scores_mean + test_scores_std, alpha=0.2)

ax.set_xlabel('Number of training examples')
ax.set_ylabel('AUROC')

ax.legend(loc='lower right', frameon=False)

sns.despine()

plt.savefig('learning_curve.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()